In [1]:
import pandas as pd
import json 
import numpy as np
from collections import OrderedDict

In [2]:
json_compound = "additional_data/ethereum_compound.json"

In [3]:
df = pd.read_csv("additional_data/Total market.csv")
df["2% Depth"] = pd.to_numeric(df["2% Depth"].str.replace(",","").str.replace("$",""))
df["-2% Depth"] = pd.to_numeric(df["-2% Depth"].str.replace(",","").str.replace("$",""))
df.head(2)

def get_total_market_1(x):
    return x*1.02*100

def get_total_market_2(x):
    return x*0.98*100

df["Total market 1"] = df["2% Depth"].apply(get_total_market_2)
df["Total market 2"] = df["-2% Depth"].apply(get_total_market_1)
df["Total market"] = (df["Total market 1"] + df["Total market 2"])/2

total_market = df[df.Pair.str.contains("/US")]["Total market"].sum()

def get_slippage_dollars(x,market= total_market):
    return 1/(1+2*x/market)

/tmp/ipykernel_4854/2673669588.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["2% Depth"] = pd.to_numeric(df["2% Depth"].str.replace(",","").str.replace("$",""))
/tmp/ipykernel_4854/2673669588.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["-2% Depth"] = pd.to_numeric(df["-2% Depth"].str.replace(",","").str.replace("$",""))


In [5]:
RATIO_COMPOUND_INDUSTRY = 0.1
LIQUIDATION_THESHOLD = 1


eth_compound = json.load(open(json_compound))
users_with_markets = [i for i in eth_compound["users"] if len(i["markets"])>0]#clean empty users

m_status = json.load(open("additional_data/compmound_markets_status.json"))
m_dict = dict(zip([i  for i in m_status], [m_status[i]["symbol"] for i in m_status]))

# get the ratio of collateral that ETH represents
collaterals = []
values = []
for k in m_dict.keys(): 
    users_with_markets = [i for i in eth_compound["users"] if len(i["markets"])>0]
    collaterals.append(m_dict[k])
    values.append(np.sum([i["markets"][0]["collateral"] for i in users_with_markets if i["markets"][0]["market"]==k ]))
df_collaterals = pd.DataFrame({"collateral": collaterals, "value": values})
df_collaterals_ns = df_collaterals[~df_collaterals.collateral.isin(["DAI","USDC","USDT","TUSD"])]
df_collaterals_ns.value = df_collaterals_ns.value/df_collaterals_ns.value.sum()

ratio_eth = df_collaterals_ns[df_collaterals_ns.collateral=="ETH"].values[0][1]

# discount stablecoins
stable_coins_markets = ["0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643","0x39AA39c021dfbaE8faC545936693aC917d5E7563",
                        "0xf650C3d88D12dB855b8bf7D11Be6C55A4e07dCC9","0x12392F67bdf24faE0AF363c24aC620a2f67DAd86"]

def get_stable_coins (x):
    return np.sum([i["collateral"] for i in x["markets"] if i["market"] in stable_coins_markets])


users_with_markets_and_stable_collat = [
    {**user_data, **{"stable_collateral": get_stable_coins(user_data)}} 
    for user_data in users_with_markets
]

users_with_markets_and_stable_collat_with_liquidation = [
    {**user_data, **{"liquidation_value": (user_data["debt"]-(user_data["stable_collateral"]))/LIQUIDATION_THESHOLD}} 
    for user_data in users_with_markets_and_stable_collat
]

users_with_markets_and_stable_collat_with_liquidation = [u for u in users_with_markets_and_stable_collat_with_liquidation if u["liquidation_value"]>0]

def get_liquidation_percentage(x, y):
    return y/x

users_with_markets_and_stable_collat_with_liquidation = [
    {**user_data, **{"liquidation_perc": get_liquidation_percentage(user_data["collateral"]-user_data["stable_collateral"],user_data["liquidation_value"])}} 
    for user_data in users_with_markets_and_stable_collat_with_liquidation
]

df_users = pd.DataFrame(users_with_markets_and_stable_collat_with_liquidation)
df_users['eth_collateral'] = df_users['collateral'] - df_users["stable_collateral"]
df_users = df_users[(df_users.eth_collateral>0)&(df_users.netValue>0)].sort_values("liquidation_perc",ascending=False)

df_users['total_liquidation'] = df_users['eth_collateral'].cumsum()*ratio_eth # the amount of ETH that will be liquidated and sold is 80% of the collateral
df_users["price_change"] = 1 - df_users["liquidation_perc"]

df_users['liquidation_slippage'] = get_slippage_dollars(df_users['total_liquidation']/RATIO_COMPOUND_INDUSTRY,total_market) # tenemos que añadir la proporcion de esto del mercado 
df_users['price_after_slippage'] = df_users['liquidation_perc']*df_users['liquidation_slippage']

df_users[["price_change","total_liquidation","liquidation_slippage"]].to_json("output/slippage_curves.json")

/tmp/ipykernel_4854/360321197.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collaterals_ns.value = df_collaterals_ns.value/df_collaterals_ns.value.sum()
/tmp/ipykernel_4854/360321197.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  return y/x
